# import

In [ ]:
import numpy as np
import pandas as pd
import pandas.tseries.offsets as offsets
import datetime as dt
import time
import importlib
import logging
from retry import retry
#import traceback
#from retrying import retry
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, Float, Text
# from sqlalchemy.types import Integer
# from sqlalchemy.types import Text

import stock

In [ ]:
importlib.reload(stock)

# MySQLに接続

In [ ]:
sql = stock.sql()

# 上場一覧から各種テーブルを作成 (メインはyahoo_stock_table)

In [ ]:
# 東証のエクセルファイルを読み込む
all_stock_table = pd.read_excel('/Users/Really/Stockyard/_dl_data/data_j.xls') # http://www.jpx.co.jp/markets/statistics-equities/misc/01.html
all_stock_table.columns = ['date', 'code', 'name', 'market', 'code_33', 'category_33', 'code_17', 'category_17', 'code_scale', 'scale'] # 列名を変更

In [ ]:
# 上場一覧のテーブル保存
all_stock_table.to_csv('/Users/Really/Stockyard/_csv/all_stock_table.csv')
sql.write_table('all_stock_table', all_stock_table)
# all_stock_table.to_sql('all_stock_table', engine, if_exists='replace')

In [ ]:
# PRO Marketを除いたテーブルの作成 (YahooにはPRO Marketのデータはない)
yahoo_stock_table = all_stock_table.ix[~all_stock_table['market'].str.contains('PRO Market')].reset_index(drop=True)
yahoo_stock_table

In [ ]:
# marketの種別で集計
yahoo_stock_table.groupby('market').count()

In [ ]:
# PRO Marketを除いたテーブルの保存
yahoo_stock_table.to_csv('/Users/Really/Stockyard/_csv/yahoo_stock_table.csv')
sql.write_table('yahoo_stock_table', yahoo_stock_table)
# yahoo_stock_table.to_sql('yahoo_stock_table', engine, if_exists='replace')

In [ ]:
# 内国株のテーブル作成
domestic_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('内国株')].reset_index(drop=True)

In [ ]:
# 内国株のテーブル保存
domestic_stock_table.to_csv('/Users/Really/Stockyard/_csv/domestic_stock_table.csv')
sql.write_table('domestic_stock_table', domestic_stock_table)
# domestic_stock_table.to_sql('domestic_stock_table', engine, if_exists='replace')

In [ ]:
# 内国株以外のテーブル作成
ex_stock_table = all_stock_table[~all_stock_table['code'].isin(domestic_stock_table['code'])].reset_index(drop=True)

In [ ]:
# 内国株以外のテーブル保存
ex_stock_table.to_csv('/Users/Really/Stockyard/_csv/ex_stock_table.csv')
sql.write_table('ex_stock_table', ex_stock_table)
# ex_stock_table.to_sql('ex_stock_table', engine, if_exists='replace')

In [ ]:
all_stock_table

In [ ]:
domestic_stock_table

In [ ]:
ex_stock_table

In [ ]:
pd.DataFrame([sql.read_table('all_stock_table').dtypes, sql.read_table('domestic_stock_table').dtypes, sql.read_table('ex_stock_table').dtypes],
            index=['all', 'domestic', 'ex'])

# 上場一覧の更新

## TODO  
__次回12月分を処理する時にやり方を整理する__  
__new_stock_table を他のカーネルやスクリプトからも参照できるようにする__

In [ ]:
# 旧テーブルの読み込み
all_stock_table_1710 = pd.read_excel('/Users/Really/Stockyard/_dl_data/data_j_1710.xls') # http://www.jpx.co.jp/markets/statistics-equities/misc/01.html
all_stock_table_1710.columns = ['date', 'code', 'name', 'market', 'code_33', 'category_33', 'code_17', 'category_17', 'code_scale', 'scale'] # 列名を変更
all_stock_table_1710

In [ ]:
# 旧テーブルの別名保存
all_stock_table_1710.to_csv('/Users/Really/Stockyard/_csv/all_stock_table_1710.csv')
sql.write_table('all_stock_table_1710', all_stock_table)
# all_stock_table_1710.to_sql('all_stock_table_1710', engine, if_exists='replace')

In [ ]:
# 新規銘柄のテーブル作成
new_stock_table = yahoo_stock_table[~yahoo_stock_table['code'].isin(all_stock_table_1710['code'])].reset_index(drop=True)
new_stock_table

# 上場廃止銘柄の更新

## TODO  
__次回12月分を処理する時に上場廃止銘柄のテーブル書き込みを追記仕様に書き換える__

In [ ]:
# 上場廃止銘柄のテーブル作成
discontinued_stock_table = all_stock_table_1710[~all_stock_table_1710['code'].isin(yahoo_stock_table['code'])].reset_index(drop=True)
# discontinued_stock_table = discontinued_stock_table.ix[~discontinued_stock_table['market'].str.contains('PRO Market')].reset_index(drop=True)
discontinued_stock_table

In [ ]:
# 上場廃止銘柄のテーブル保存
discontinued_stock_table.to_csv('/Users/Really/Stockyard/_csv/discontinued_stock_table.csv')
sql.write_table('discontinued_stock_table', all_stock_table)
# discontinued_stock_table.to_sql('discontinued_stock_table', engine, if_exists='replace')

# 連続読み込み用コードリストの作成例

## yahoo 連続読み込み用コードリスト作成

In [ ]:
start_index = 3100
increase_number = 100
end_index = start_index + increase_number

reading_code = sql.get_yahoo_stock_code(start_index, end_index)
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))

## 新規銘柄読み込み用コードリスト作成

In [ ]:
reading_code = list(new_stock_table['code'])
reading_code

# クラス不使用版コード

## MySQLに接続

In [ ]:
db_settings = {
    "host": 'localhost',
    "database": 'StockPrice_Yahoo_1',
    "user": 'user',
    "password": 'password',
    "port":'3306'
}
engine = create_engine('mysql://{user}:{password}@{host}:{port}/{database}'.format(**db_settings))

## 読み込む内国株のコードリスト作成 (クラス不使用版)

In [ ]:
# 内国株だけにする
# MySQLに保存済みの内国株テーブルから作成。今後はこちらを使用する
yahoo_stock_table = pd.read_sql_table('yahoo_stock_table', engine, index_col=None).drop('index', axis=1)

start_index = 2810
increase_number = 10
end_index = start_index + increase_number

reading_code = list(yahoo_stock_table['code'][start_index : end_index])
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))